In [ ]:
!pip install google-generativeai

In [2]:
import google.generativeai as genai
import pandas as pd
import os
import time
import logging
from google.api_core import exceptions

# --- 設定項目 ---
# APIキーの設定
from google.colab import userdata
API_KEY = userdata.get('GEMINI_API_KEY')

BASE_DIR = 'https://github.com/naoki-kokaze/British_Warship_Career/tree/main/'
INPUT_FILE_PATH = os.path.join(BASE_DIR, 'strata/SL1.xlsx') #１等戦列艦を例に
OUTPUT_DIR = os.path.join(BASE_DIR, 'api_out/SL1')
LOG_FILE = os.path.join(BASE_DIR, 'log/generation.log')

# API設定
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-2.5-pro')

# ロギング設定
logging.basicConfig(filename=LOG_FILE, level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# --- プロンプトテンプレート ---
prompt_template = """
あなたは知識グラフの専門家です。提供されたコンテキスト情報（オントロジーの抜粋、モデリングルール）と、特定のイギリス海軍艦船に関するキャリア記述テキストに基づいて、その艦船のキャリアをRDF Turtle形式で記述してください。

# コンテキスト情報

## 1. 使用するオントロジーと接頭辞
@prefix : <http://www.example.com/ontology/warship#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix crm: <http://www.cidoc-crm.org/cidoc-crm/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix sealit: <http://www.sealitproject.eu/ontology/> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix myData: <http://www.example.com/myData/> .

## 2. 主要なクラスとプロパティ (warship_career_ontology_v3.ttl の内容を要約)
- 艦船: `sealit:Ship`
- イベント: `crm:E12_Production`, `crm:E13_Attribute_Assignment`, `:Commissioning`, `:Decommissioning`, `:Recommissioning`, `sealit:ShipRepair` (サブクラス含む), `:ShipConversion` (サブクラス含む), `:Capture`, `crm:E6_Destruction`
- 状態/期間: `crm:E93_Presence`, `crm:E52_Time-Span`
- 分類/型の語彙運用方針
    - 【固定語彙。新規個体は作らない】`:OperationalStatus` (`:InCommission`, `:InOrdinary`, `:PaidOff`,　`:UnderFitting`, `:InNonCombatDuty`, `:UnderRepair`, `:BeingBrokenUp`)
    - 【固定語彙】`:ShipRating`(`:FirstRate`, `:SecondRate`, `:ThirdRate`, `:ForthRate`, `:FifthRate`, `:SixthRate`)
    - 【固定語彙】`:PropulsionType`(`:Sailing`, `:PaddleWheel`, `:ScrewPropeller`)
    - `:ShipRole`(`:HospitalShip`, `:SheerHulk`など、テキストに登場する情報をインスタンスとしてad hocに記述し（※ただし重複不可）、後でホワイトリストを作成して表記を整える)
    - `:ShipType`(`:Frigate`, `:Cutter`, `:ShipOfTheLine`など軍艦としての機能を表す。記述方針は`:ShipRole`と同様)
- その他: `crm:E53_Place`, `crm:E97_Monetary_Amount`, `crm:E98_Currency`
- 主要プロパティ: `crm:P4_has_time-span`, `crm:P7_took_place_at`, `:modifies_ship`, `:incurred_cost`, `crm:P21_had_general_purpose`, `crm:P13_destroyed`, `crm:P140_assigned_attribute_to`, `crm:P141_assigned`, `crm:P164_is_temporally_specified_by`, `crm:P167_was_within`, `crm:P195_was_a_presence_of`, `crm:P195i_had_presence`,`:has_operational_status`, `:has_ship_rating`, `:has_ship_role`, `:has_propulsion_type`, `:has_ship_type`, `crm:P182i_starts_after_or_with_the_end_of`, `crm:P182_ends_before_or_with_the_start_of`, `crm:P90_has_value`, `crm:P180_has_currency`, `dcterms:source`, `rdfs:label`, `rdfs:comment`

## 3. モデリングルール (最重要)
### 必須のアンカーイベント
- **キャリアの始点**: 艦船のキャリアは基本的に「建造」イベント (`crm:E12_Production`) から始まる。このイベントの最初の日付（`P79`または`P82`）あるいは進水日（`P80`）は、イベントヒストリー分析で `age` を計算するために **【最重要】** である。
- **キャリアの終点**: 艦船のキャリアの終わり（売却、解体、難破など）は、`crm:E8_Acquisition`（売却）または `crm:E6_Destruction`（解体・難破）として **【必ず】** 記述する。このイベントの日付が、EHA分析の「キャリア終了」イベントの発生日となる。
- **オントロジーファイルにないクラス/プロパティは LLM/API で新設しない**
- エンティティ命名規則: 艦船ID、イベントID、Presence IDは、出典で用いられる「艦種カテゴリコード」（例: `SL3`, `SBS`, `SSL`など）をプレフィックスとして使用する。
    - 艦船: `myData:{{ship_type_code}}_{{連番}}` (例: `myData:SL3_0027`)
    - イベント: `myData:event_{{ship_type_code}}_{{連番}}_{{通番}}` (例: `myData:event_SL3_0027_01`)
    - Presence: `myData:presence_{{ship_type_code}}_{{連番}}_{{通番}}` (例: `myData:presence_SL3_0027_01`)
    - **この命名規則は、イベントヒストリー分析で「艦種」という要因を特定するために 【非常に重要】 である。**
    - 場所 `myData:place_{{連番}}` (既知は再利用)

- 「艦種カテゴリコード」の内訳は次の通り。以下、特記事項がない場合の:has_propulsion_typeの目的語と、:has_ship_typeの目的語も併記する
    - SL1: 1等戦列艦、:has_propulsion_type :Sailing; :has_ship_type :ShipOfTheLine
    - SL2: 2等戦列艦、:has_propulsion_type :Sailing; :has_ship_type :ShipOfTheLine
    - SL3: 3等戦列艦、:has_propulsion_type :Sailing; :has_ship_type :ShipOfTheLine
    - SL4: 4等戦列艦、:has_propulsion_type :Sailing; :has_ship_type :ShipOfTheLine
    - SF5: 5等戦列艦、:has_propulsion_type :Sailing; :has_ship_type :SailingFrigate
    - SC6: 6等戦列艦、:has_propulsion_type :Sailing; :has_ship_type :Corvette
    - SSL: 帆走スループ艦、:has_propulsion_type :Sailing; :has_ship_type :SailingSloop
    - SBS: ブリッグスループ艦、:has_propulsion_type :Sailing; :has_ship_type :BrigsSloop
    - SCS: カッター・スクーナー艦、:has_propulsion_type :Sailing; :has_ship_type :CuttersAndSchooners
    - SM: その他帆走艦、:has_propulsion_type :Sailing; :has_ship_type :SailingMisc
    - 以下、外輪推進艦、:has_propulsion_type :PaddleWheel
        - PSPV: Steam Paddle Vessel、:has_ship_type :SteamPaddleVessel
        - PF: Paddle Frigate、:has_ship_type :PaddleFrigate
        - PSL: Paddle Sloop、:has_ship_type :PaddleSloop
        - PGV: Paddle Gunvessel、:has_ship_type :PaddleGunvessel
        - PM: Paddle Misc、:has_ship_type :PaddleMisc
    - 以下、スクリュー推進艦、:has_propulsion_type :ScrewPropeller; :has_ship_type :ShipOfTheLine
        - SCSL: Screw Sloop、:has_ship_type :ScrewSloop
        - SCGVG: Screw Gunvessel、:has_ship_type :ScrewGunvessel
        - SCM: Screw Misc、:has_ship_type :ScrewMisc
    - 以下、装甲艦
        - AIF: Armoured Iron Frigate、:has_ship_type :ArmouredIronFrigate
        - AWC: Armoured Wooden Cruiser、:has_ship_type :ArmouredWoodenCruiser
        - ACD: Armoured Coastal Defence Ship、:has_ship_type :ArmouredCoastalDefenceShip

- 役割/タイプ排他: `:has_ship_role` (具体的役割がある場合) **または** `:has_ship_type` (役割がない場合) のどちらか一方のみ付与。
    - **役割/タイプの記述**: `:has_ship_role`（例: `:ConvictHulk`）は `Presence` が `:InNonCombatDuty` の期間に、その具体的な役割を記述するために使用する。`:has_ship_type`（例: `:ShipOfTheLine`）は、艦船（`sealit:Ship`）の基本的な分類を記述するために使用する。
- イベント抽出: テキストから主要イベントを抽出し適切なクラスで記述。
    - 戦闘関連= :CombatCapabilityInvestment／非戦闘転用= :NonCombatDutyInvestment を、該当するイベント（Commissioning/Refit/各種Repair/Conversion など）のcrm:P21_had_general_purposeに付与。**`:incurred_cost`プロパティを伴うイベントの場合は、crm:P21を必ず併記する。**
    - E13_Attribute_Assignment は「等級変更／改名など属性変更」に限定

### Presence（状態）の厳密な定義
- **状態の網羅**: 艦船のキャリアは、「始点」（建造完了）から「終点」（売却・解体）まで、**一切の隙間なく**、連続した `crm:E93_Presence` でカバーされなければならない。
- **状態の必須定義**: すべての `Presence` は、`:has_operational_status` プロパティを **【必ず1つだけ】** 持たなければならない（`:InCommission`, `:InOrdinary`, `:PaidOff`, `:UnderFitting`, `:InNonCombatDuty`, `:UnderRepair`, `:BeingBrokenUp` のいずれか）。
- **厳密な分割**: `:operational_status` が変更される（例: `:InOrdinary` から `:InCommission` へ）場合、その間には **【必ず】** イベント（例: `:Commissioning`）が存在し、`Presence` はそのイベントを境に分割されなければならない。
- **修理の独立**: テキストに「修理 (Repair)」や「改装 (Refit)」が記載された期間は、他の状態（`:InCommission`や`:InOrdinary`）から独立させ、`:UnderRepair` の `Presence` として記述する。
- すべての Presence は当該艦に P195 で結節
- Presence属性:
    - `:has_operational_status`: 必ず1つだけ付与 (`:InCommission`, `:InOrdinary`, `:UnderRepair`, `:InNonCombatDuty`)。
    - `:has_propulsion_type`: 常に付与 (例: `:Sailing`)。
    - `:has_ship_rating`: 付与。等級変更は`E13`イベントを生成し、以降のPresenceで反映。
    - 役割/タイプ排他: `:has_ship_role` (具体的役割がある場合) **または** `:has_ship_type` (役割がない場合) のどちらか一方のみ付与。
- 時間情報: `Time-Span`に詳細な日付 (`xsd:date`, `xsd:gYearMonth`, `xsd:gYear`) を記述。連続するPresenceはイベントを介して`crm:P182i_starts_after_or_with_the_end_of`と`crm:P182_ends_before_or_with_the_start_of`で接続。
    - 日付が1点で与えられる場合は crm:P82_at_some_time_within、範囲の場合は crm:P79_beginning_is_qualified_by／crm:P80_end_is_qualified_by を使用
- 場所情報: `crm:P7_took_place_at`/`crm:P167_was_within`で`myData:place_XXXX`にリンク。
    - 命名規則で 場所は既知 IRI を再利用（myData:place_XXXX）し、重複生成を避ける
- その他:
    - `sealit:Ship` インスタンスには **必ず** `dcterms:source` プロパティを追加し、出典情報（例: "Winfield, Rif. British Warships in the Age of Sail, 1817-1863. Seaforth Publishing, 2014."）をリテラル値として記述してください。**このトリプルは必ず艦船インスタンス定義の一部として記述し、ファイル末尾などに単独で記述しないでください。**
    - `rdfs:label` (英)。`rdfs:comment` (補足)。

### イベントと費用の厳格なモデリングルール

**(1) イベントの定義**:
   - テキストから抽出される主要な出来事（建造、修理、改装、就役、退役など）は、**必ず** `crm:E7_Activity` またはそのサブクラス（例: `crm:E12_Production`, `:Refit`）として定義されなければなりません。

**(2) 費用情報のリンク（最重要）**:
   - テキストに費用（例: "£25,352"）が記載されている場合、その費用は **【絶対に】** それ自体を `crm:E97_Monetary_Amount` として型付けたエンティティ（例: `myData:event_... a crm:E97_Monetary_Amount`）として定義してはなりません。
   - 費用（`crm:E97_Monetary_Amount`）は、**必ず**、上記 1. で定義された**イベント**（`crm:E7_Activity` 等）から `:incurred_cost` プロパティを使ってリンクされる**無名ノード（Blank Node, `[]`）**として記述しなければなりません。

**(3) 具体的な禁止例と必須例**:

   - **【誤った記述 - このパターンは絶対に禁止】**:
     ```turtle
     # 誤り: Monetary_Amount をトップレベルのイベントIDで定義している
     myData:event_SL1_0041_01_hull a crm:E97_Monetary_Amount ;
       rdfs:comment "Hull cost as part of first cost"@en ;
       crm:P90_has_value "107731.0"^^xsd:decimal .

     ```
    - **【正しい記述 - このパターンを必須とする】**:
     ```turtle
     # 正解: :Fitting イベントから :incurred_cost で無名ノードをリンクしている
     myData:event_SL1_0015_18 a :Fitting ;
       rdfs:label "Fitted at Portsmouth (6.1806-9.1807)"@en ;
       :modifies_ship myData:SL1_0015 ;
       :incurred_cost [
           a crm:E97_Monetary_Amount ;
           rdfs:comment "£25,352"@en ;
           crm:P90_has_value "25352.0"^^xsd:decimal
       ] .
     ```

**(4) P21との併記（既存ルールの再強調）**:
   - `:incurred_cost`プロパティを伴うイベント（`crm:E7_Activity` 等）には、**必ず** `crm:P21_had_general_purpose`（`:CombatCapabilityInvestment` または `:NonCombatDutyInvestment`）も併記すること。

## 4. 解釈とモデリングの「模範例」

- 以下は、原文テキストをどのように解釈し、ルールに基づいてRDF（特にイベントとPresenceの連鎖）を構築するかの「模範例」である。この例のパターンに厳密に従うこと。

# === (模範例) 元となるテキストデータ ===
  Vanguard Pembroke Dyd
  As built: 190ft Din, 155ft Din x 57ft Din oa (56ft 3in for tonnage) x 23ft 4in. 2,608621,.bm.
  Ord: 29.6.1832. K: 5.1833. L: 25.8.1835. C: 19.9.1835-5.7.1836 at Portsmouth.
  First cost: £56,983 (+ fitting £20,756).
  Commissioned 18.3.1836 under Capt. Duncombe Pleydell Bouverie, for the Mediterranean. On 25.1.1837 under Capt. Sir Thomas Fellowes, still in the Mediterranean. On 2.4.1840 under Capt. Sir David Dunn; refitted at Portsmouth
  (for £14,931) 4-9.1840, then to the Mediterranean: operations on the Syrian coast 1840; subsequently off Lisbon, then home in 1842; paid off 8.1843. Fitted for sea at Plymouth (for £7,693) 9.1844-5.1845.
  Recommissioned 4.2.1845 under Capt. George Wickens Willes (died 26.10.1847), for the Channel Squadron, then to Experimental Squadron; to Lisbon 1847. On 6.11.1847 under Capt. George Frederick Rich; in the
  Mediterranean 1848; paid off 28.3.1849. Small Repair at Plymouth (for £20,004) 5.1849- 4.1850. On 18.2.1861 under Capt. Edmund Heathcote, as guard ship at Kingstown; laid up at Sheerness 3.1862. Renamed Ajax 20.10.1867. BU at Chatham completed 26.6.1875.

# === (模範例) 理想のRDFデータ ===
@prefix : <http://www.example.com/ontology/warship#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix crm: <http://www.cidoc-crm.org/cidoc-crm/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix sealit: <http://www.sealitproject.eu/ontology/> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix myData: <http://www.example.com/myData/> .

### Places
myData:place_portsmouth a crm:E53_Place ; rdfs:label "Portsmouth"@en .
myData:place_syrian_coast a crm:E53_Place ; rdfs:label "Syrian coast"@en .
myData:place_lisbon     a crm:E53_Place ; rdfs:label "Lisbon"@en .
myData:place_plymouth   a crm:E53_Place ; rdfs:label "Plymouth"@en .
myData:place_kingstown  a crm:E53_Place ; rdfs:label "Kingstown"@en .
myData:place_sheerness  a crm:E53_Place ; rdfs:label "Sheerness"@en .
myData:place_chatham    a crm:E53_Place ; rdfs:label "Chatham"@en .

### Ship
myData:SL2_0027 a sealit:Ship ;
  rdfs:label "HMS Vanguard"@en ;
  :has_ship_rating :SecondRate ;
  :has_propulsion_type :Sailing ;
  :has_ship_type :ShipOfTheLine ;
  dcterms:source "Winfield, Rif. British Warships in the Age of Sail, 1817-1863. Seaforth Publishing, 2014."@en .

### Construction and initial fitting
myData:event_SL2_0027_01 a crm:E12_Production ;
  rdfs:label "As built: 190ft 0in, 155ft 0in x 57ft 0in oa (56ft 3in for tonnage) x 23ft 4in. 2,608.bm."@en ;
  crm:P4_has_time-span [
    a crm:E52_Time-Span ;
    crm:P79_beginning_is_qualified_by "1832-06-29"^^xsd:date ; # Ord
    crm:P80_end_is_qualified_by       "1835-08-25"^^xsd:date   # L
  ] ;
  :incurred_cost [
    a crm:E97_Monetary_Amount ;
    rdfs:comment "First cost: £56,983 (+ fitting £20,756)"@en ;
    crm:P90_has_value "77739.0"^^xsd:decimal
  ] .

### Commissioning and service periods
myData:event_SL2_0027_02 a :Commissioning ;
  rdfs:label "Commissioned 19.9.1835"@en ;
  crm:P4_has_time-span [
    a crm:E52_Time-Span ;
    crm:P79_beginning_is_qualified_by "1835-09-19"^^xsd:gYearMonth ;
    crm:P80_end_is_qualified_by       "1836-07-05"^^xsd:gYearMonth
  ] .

myData:event_SL2_0027_03 a :Commissioning ;
  rdfs:label "Commissioned 18.3.1836"@en ;
  crm:P4_has_time-span [ a crm:E52_Time-Span ; crm:P79_beginning_is_qualified_by "1836-03-18"^^xsd:date ] .

myData:event_SL2_0027_04 a :Commissioning ;
  rdfs:label "Commissioned 25.1.1837"@en ;
  crm:P4_has_time-span [ a crm:E52_Time-Span ; crm:P79_beginning_is_qualified_by "1837-01-25"^^xsd:date ] .

myData:event_SL2_0027_05 a :Commissioning ;
  rdfs:label "Commissioned 2.4.1840"@en ;
  crm:P4_has_time-span [ a crm:E52_Time-Span ; crm:P79_beginning_is_qualified_by "1840-04-02"^^xsd:date ] .

myData:presence_SL2_0027_01 a crm:E93_Presence ;
  rdfs:label "From her first commissioning to Refit 4-9.1840"@en ;
  crm:P195i_had_presence myData:SL2_0027 ;
  :has_operational_status :InCommission ;
  crm:P167_was_within myData:place_portsmouth ;
  crm:P164_is_temporally_specified_by [ a crm:E52_Time-Span ;
    crm:P182i_starts_after_or_with_the_end_of myData:event_SL2_0027_02 ;
    crm:P182_ends_before_or_with_the_start_of myData:event_SL2_0027_06 ] .

myData:event_SL2_0027_06 a :Refit ;
  rdfs:label "Refitted at Portsmouth (4-9.1840) for £14,931"@en ;
  :modifies_ship myData:SL2_0027 ;
  crm:P7_took_place_at myData:place_portsmouth ;
  crm:P4_has_time-span [
    a crm:E52_Time-Span ;
    crm:P79_beginning_is_qualified_by "1840-04"^^xsd:gYearMonth ;
    crm:P80_end_is_qualified_by       "1840-09"^^xsd:gYearMonth
  ] ;
  :incurred_cost [
    a crm:E97_Monetary_Amount ;
    rdfs:comment "£14,931"@en ;
    crm:P90_has_value "14931.0"^^xsd:decimal
  ] ;
  crm:P21_had_general_purpose :CombatCapabilityInvestment .

### Presence during Syrian operations
myData:presence_SL2_0027_02 a crm:E93_Presence ;
  rdfs:label "Operations on the Syrian coast (1840), Off Lisbon (1841-1842)"@en ;
  crm:P195i_had_presence myData:SL2_0027 ;
  :has_operational_status :InCommission ;
  crm:P7_took_place_at myData:place_syrian_coast, myData:place_lisbon ;
  crm:P164_is_temporally_specified_by [ a crm:E52_Time-Span ;
    crm:P182i_starts_after_or_with_the_end_of myData:event_SL2_0027_06 ;
    crm:P182_ends_before_or_with_the_start_of myData:event_SL2_0027_07 ] .

myData:event_SL2_0027_07 a :Decommissioning ;
  rdfs:label "Paid off 8.1843"@en ;
  crm:P4_has_time-span [ a crm:E52_Time-Span ; crm:P79_beginning_is_qualified_by "1843-08"^^xsd:gYearMonth ] .

myData:event_SL2_0027_08 a :FittingForSea ;
  rdfs:label "Fitting for sea at Plymouth (9.1844-5.1845) for £7,693"@en ;
  :modifies_ship myData:SL2_0027 ;
  crm:P7_took_place_at myData:place_plymouth ;
  crm:P4_has_time-span [
    a crm:E52_Time-Span ;
    crm:P79_beginning_is_qualified_by "1844-09"^^xsd:gYearMonth ;
    crm:P80_end_is_qualified_by       "1845-05"^^xsd:gYearMonth
  ] ;
  :incurred_cost [
    a crm:E97_Monetary_Amount ;
    rdfs:comment "£7,693"@en ;
    crm:P90_has_value "7693.0"^^xsd:decimal
  ] ;
  crm:P21_had_general_purpose :CombatCapabilityInvestment .

myData:presence_SL2_0027_03 a crm:E93_Presence ;
  rdfs:label "Between Paid off and Fitting"@en ;
  crm:P195i_had_presence myData:SL2_0027 ;
  :has_operational_status :PaidOff ;
  crm:P164_is_temporally_specified_by [ a crm:E52_Time-Span ;
    crm:P182i_starts_after_or_with_the_end_of myData:event_SL2_0027_07 ;
    crm:P182_ends_before_or_with_the_start_of myData:event_SL2_0027_08 ] .

myData:event_SL2_0027_09 a :Recommissioning ;
  rdfs:label "Recommissioned 4.2.1845"@en ;
  crm:P4_has_time-span [
    a crm:E52_Time-Span ;
    crm:P79_beginning_is_qualified_by "1845-02-04"^^xsd:date ;
    crm:P80_end_is_qualified_by       "1847"^^xsd:gYearMonth
  ] .

myData:presence_SL2_0027_04 a crm:E93_Presence ;
  rdfs:label "Between Fitting and Recommissioning"@en ;
  crm:P195i_had_presence myData:SL2_0027 ;
  :has_operational_status :UnderFitting ;
  crm:P164_is_temporally_specified_by [ a crm:E52_Time-Span ;
    crm:P182i_starts_after_or_with_the_end_of myData:event_SL2_0027_08 ;
    crm:P182_ends_before_or_with_the_start_of myData:event_SL2_0027_09 ] .

myData:event_SL2_0027_10 a :Commissioning ;
  rdfs:label "On 6.11.1847 under Capt. George Frederick Rich; in the Mediterranean 1848"@en ;
  crm:P4_has_time-span [
    a crm:E52_Time-Span ;
    crm:P79_beginning_is_qualified_by "1847-11-06"^^xsd:date ;
    crm:P80_end_is_qualified_by       "1848"^^xsd:gYearMonth
  ] .

myData:presence_SL2_0027_05 a crm:E93_Presence ;
  rdfs:label "In Commission (4.2.1845–28.3.1849) for the Channel Squadron, Experimental Squadron, Mediterranean"@en ;
  crm:P195i_had_presence myData:SL2_0027 ;
  :has_operational_status :InCommission ;
  crm:P164_is_temporally_specified_by [ a crm:E52_Time-Span ;
    crm:P182i_starts_after_or_with_the_end_of myData:event_SL2_0027_09 ;
    crm:P182_ends_before_or_with_the_start_of myData:event_SL2_0027_11 ] .

myData:event_SL2_0027_11 a :Decommissioning ;
  rdfs:label "Paid off 28.3.1849"@en ;
  crm:P4_has_time-span [ a crm:E52_Time-Span ; crm:P82_at_some_time_within "1849-03-28"^^xsd:date ] .

myData:presence_SL2_0027_05 a crm:E93_Presence ;
  rdfs:label "Between Decommissioning (28.3.1849) and the beginning of small repair (5.1849)"@en ;
  crm:P195i_had_presence myData:SL2_0027 ;
  :has_operational_status :PaidOff ;
  crm:P7_took_place_at myData:place_plymouth ;
  crm:P164_is_temporally_specified_by [ a crm:E52_Time-Span ;
    crm:P182i_starts_after_or_with_the_end_of myData:event_SL2_0027_11 ;
    crm:P182_ends_before_or_with_the_start_of myData:event_SL2_0027_12 ] .

myData:event_SL2_0027_12 a :SmallRepair ;
  rdfs:label "Small Repair at Plymouth (5.1849–4.1850) for £20,004"@en ;
  sealit:repaired myData:SL2_0027 ;
  crm:P7_took_place_at myData:place_plymouth ;
  crm:P4_has_time-span [
    a crm:E52_Time-Span ;
    crm:P79_beginning_is_qualified_by "1849-05"^^xsd:gYearMonth ;
    crm:P80_end_is_qualified_by       "1850-04"^^xsd:gYearMonth
  ] ;
  :incurred_cost [
    a crm:E97_Monetary_Amount ;
    rdfs:comment "£20,004"@en ;
    crm:P90_has_value "20004.0"^^xsd:decimal
  ] ;
  crm:P21_had_general_purpose :CombatCapabilityInvestment .

myData:presence_SL2_0027_06 a crm:E93_Presence ;
  rdfs:label "During the small repair"@en ;
  crm:P195i_had_presence myData:SL2_0027 ;
  :has_operational_status :UnderRepair ;
  crm:P7_took_place_at myData:place_plymouth ;
  crm:P164_is_temporally_specified_by [ a crm:E52_Time-Span ;
    crm:P79_beginning_is_qualified_by "1849-05"^^xsd:gYearMonth ;
    crm:P80_end_is_qualified_by       "1850-04"^^xsd:gYearMonth ] .

myData:presence_SL2_0027_07 a crm:E93_Presence ;
  rdfs:label "[Assumption] Between the end of small repair (4.1850) and commissioning as Guard ship at Kingstown (18.2.1861)"@en ;
  crm:P195i_had_presence myData:SL2_0027 ;
  :has_operational_status :InOrdinary ;
  crm:P7_took_place_at myData:place_plymouth ;
  crm:P164_is_temporally_specified_by [ a crm:E52_Time-Span ;
    crm:P182i_starts_after_or_with_the_end_of myData:event_SL2_0027_12 ;
    crm:P182_ends_before_or_with_the_start_of myData:event_SL2_0027_13 ] .

myData:event_SL2_0027_13 a :Recommissioning ;
  rdfs:label "Recommissioned 18.2.1861"@en ;
  crm:P4_has_time-span [ a crm:E52_Time-Span ; crm:P82_at_some_time_within "1861-02-18"^^xsd:date ] .

myData:presence_SL2_0027_08 a crm:E93_Presence ;
  rdfs:label "Guard ship at Kingstown (18.2.1861–3.1862)"@en ;
  crm:P195i_had_presence myData:SL2_0027 ;
  :has_operational_status :InNonCombatDuty ;
  :has_ship_role :GuardShip ;
  crm:P7_took_place_at myData:place_kingstown ;
  crm:P164_is_temporally_specified_by [ a crm:E52_Time-Span ;
    crm:P182i_starts_after_or_with_the_end_of myData:event_SL2_0027_13 ;
    crm:P182_ends_before_or_with_the_start_of myData:event_SL2_0027_14 ] .

myData:event_SL2_0027_14 a :Decommissioning ;
  rdfs:label "Laid up at Sheerness 3.1862"@en ;
  crm:P4_has_time-span [ a crm:E52_Time-Span ; crm:P82_at_some_time_within "1862-03"^^xsd:gYearMonth ] .

### Renaming and Destruction
myData:event_SL2_0027_15 a crm:E13_Attribute_Assignment ;
  rdfs:label "Renamed Ajax 20.10.1867"@en ;
  crm:P140_assigned_attribute_to myData:SL2_0027 ;
  crm:P141_assigned :Name_Ajax ;
  crm:P4_has_time-span [ a crm:E52_Time-Span ; crm:P82_at_some_time_within "1867-10-20"^^xsd:date ] .

myData:event_SL2_0027_16 a crm:E6_Destruction ;
  rdfs:label "Broken up (BU) at Chatham completed 26.6.1875"@en ;
  crm:P13_destroyed myData:SL2_0027 ;
  crm:P7_took_place_at myData:place_chatham ;
  crm:P4_has_time-span [ a crm:E52_Time-Span ; crm:P80_end_is_qualified_by "1875-06-26"^^xsd:date ] .

myData:presence_SL2_0027_09 a crm:E93_Presence ;
  rdfs:label "[Assumption] After laiding up until BU"@en ;
  crm:P195i_had_presence myData:SL2_0027 ;
  :has_operational_status :InOrdinary ;
  crm:P7_took_place_at myData:place_sheerness ;
  crm:P164_is_temporally_specified_by [ a crm:E52_Time-Span ;
    crm:P182i_starts_after_or_with_the_end_of myData:event_SL2_0027_14 ;
    crm:P182_ends_before_or_with_the_start_of myData:event_SL2_0027_16 ] .

# === (模範例 終了) ===

# 入力データ

## 艦船ID
{ship_id}

## キャリア記述テキスト
```text
{career_text}

# 出力指示
- 上記のコンテキスト情報とルールに基づき、艦船 `{ship_id}` のキャリア記述テキストをRDF Turtle形式で出力してください。
- 接頭辞の定義から始めてください。
- インデントは適切に行ってください。
- 生成するのはTurtleコード本体のみとし、前後の説明文や ```turtle ... ``` のようなマークダウンは含めないでください。
- - **全ての文字列リテラル ("...") は有効なUTF-8文字列とし、Turtleの構文規則に従ってください。特に引用符 (") や特殊文字 (\) のエスケープに注意し、b'...' のようなバイト列表現は絶対に含まないでください。**

"""

# --- メイン処理 ---
def generate_turtle_data(ship_id, career_text):
    """Gemini APIを呼び出してTurtleデータを生成する関数"""
    prompt = prompt_template.format(ship_id=ship_id, career_text=career_text)
    retries = 5
    delay = 1  # 初期遅延（秒）

    generation_config = genai.types.GenerationConfig(
        temperature=0.1
    )

    for i in range(retries):
        try:
            response = model.generate_content(prompt)
            if response.parts:
                 turtle_data = response.parts[0].text
                 # ```turtle ... ```のようなマークダウンを除去する処理を追加
                 turtle_data = turtle_data.strip().removeprefix("```turtle").removesuffix("```").strip()
                 return turtle_data
            elif response.text:
                 turtle_data = response.text
                 turtle_data = turtle_data.strip().removeprefix("```turtle").removesuffix("```").strip()
                 return turtle_data
            else:
                 logging.warning(f"[{ship_id}] No text content found in response parts.")
                 return None # Or raise an error

        except exceptions.ResourceExhausted as e:
            logging.warning(f"[{ship_id}] Rate limit hit, retrying in {delay} seconds... ({i+1}/{retries})")
            time.sleep(delay)
            delay *= 2  # Exponential backoff
        except exceptions.GoogleAPIError as e:
            logging.error(f"[{ship_id}] API Error occurred: {e}. Retrying in {delay} seconds... ({i+1}/{retries})")
            time.sleep(delay)
            delay *= 2
        except Exception as e:
            logging.error(f"[{ship_id}] An unexpected error occurred: {e}")
            #予期せぬエラーの場合はリトライしない
            break # Exit retry loop for unexpected errors

    logging.error(f"[{ship_id}] Failed to generate data after {retries} retries.")
    return None

if __name__ == "__main__":
    # 出力ディレクトリ作成
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # ファイル読み込み
    try:
        df = pd.read_excel(INPUT_FILE_PATH)
        logging.info(f"Loaded {len(df)} records from {INPUT_FILE_PATH}")
    except FileNotFoundError:
        logging.error(f"CSV file not found: {INPUT_FILE_PATH}")
        exit()
    except Exception as e:
        logging.error(f"Error loading CSV: {e}")
        exit()

    # 各艦船について処理
    for index, row in df.iterrows():
        # 'ID' と 'text' 列が存在するか確認
        if 'ID' not in row or 'text' not in row:
             logging.error(f"Skipping row {index+1}: Missing 'ID' or 'text' column.")
             continue

        ship_id = row['ID']
        career_text = row['text']

        # IDやテキストが空でないか確認
        if pd.isna(ship_id) or not ship_id or pd.isna(career_text) or not career_text:
            logging.warning(f"Skipping row {index+1}: Empty ID ('{ship_id}') or text.")
            continue

        logging.info(f"Processing ship: {ship_id}")
        print(f"Processing ship: {ship_id}")

        # API呼び出し
        turtle_output = generate_turtle_data(ship_id, career_text)

        # ファイル保存
        if turtle_output:
            # IDから 'myData:' プレフィックスを除去してファイル名に使用
            filename_base = ship_id.split(':')[-1]
            output_path = os.path.join(OUTPUT_DIR, f"{filename_base}.ttl")
            try:
                with open(output_path, 'w', encoding='utf-8') as f:
                    f.write(turtle_output)
                logging.info(f"Successfully generated and saved: {output_path}")
                print(f"Successfully generated and saved: {output_path}")
            except Exception as e:
                logging.error(f"[{ship_id}] Error saving file {output_path}: {e}")
        else:
            logging.error(f"[{ship_id}] Failed to generate Turtle data.")

        # APIへの負荷軽減のため少し待機
        time.sleep(1)

    logging.info("Processing finished.")


